In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Google A2A (Agent-to-Agent) Communication

This notebook introduces you to Google's Agent-to-Agent (A2A) protocol, a standardized way for AI agents to communicate and collaborate.  

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynb">
      <img src="https://avatars.githubusercontent.com/u/33467679?s=200&v=4" width="32px" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynbb">
      <img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fai-ml-recipes%2Fmain%2Fnotebooks%2Fquickstart%2Fagent2agent%2Fa2a_quickstart.ipynb">
    <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
    Open in Colab Enterprise
    </a>
  </td>

</table>

## What You'll Build

A three-agent system that works together to analyze trending topics:
1. **Trending Topics Agent** - Searches the web for current trending topics
2. **Trend Analyzer Agent** - Performs deep analysis with quantitative data
3. **Host Agent** - Orchestrates the other agents to provide comprehensive insights

<img src="../../../docs/images/a2a-diagram.png" alt="drawing" width="1000"/>


## Prerequisites

- Python 3.11+
- Google Cloud Project with Vertex AI enabled
- Basic understanding of async Python

## Other Resources

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [A2A Protocol Specification](https://github.com/google/a2a)
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai)
- Codelabs:
  - [Google's Agent Stack in Action: ADK, A2A, MCP on Google Cloud](https://codelabs.developers.google.com/instavibe-adk-multi-agents/instructions)
  - [Getting Started with Agent-to-Agent (A2A) Protocol: Gemini on Cloud Run](https://codelabs.developers.google.com/intro-a2a-purchasing-concierge)
  - [Getting Started with MCP, ADK and A2A](https://codelabs.developers.google.com/codelabs/currency-agent)

#### Important!
A2A is a work in progress (WIP) thus, in the near future there might be changes that are different from what demonstrated in this code.

### Setup and Installation

First, let's install the required dependencies:

In [ ]:
# Install required packages
!pip3 install --upgrade -q google-genai google-adk a2a-sdk python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

## 1. Introduction to A2A

### What is Agent-to-Agent (A2A) Communication?

A2A is a standardized protocol that enables AI agents to:
- **Discover** each other's capabilities
- **Communicate** using a common JSON-RPC based protocol
- **Collaborate** to solve complex tasks
- **Stream** responses for real-time interactions

### Architecture Overview

### Environment Configuration

In [ ]:
import os
import sys

# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'PROJECT_ID'  # Replace with your project ID
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'  # Replace with your location

from dotenv import load_dotenv
load_dotenv()

print("Environment variables configured:")
print(f"GOOGLE_GENAI_USE_VERTEXAI: {os.environ['GOOGLE_GENAI_USE_VERTEXAI']}")
print(f"GOOGLE_CLOUD_PROJECT: {os.environ['GOOGLE_CLOUD_PROJECT']}")
print(f"GOOGLE_CLOUD_LOCATION: {os.environ['GOOGLE_CLOUD_LOCATION']}")

In [3]:
# Authenticate your notebook environment (Colab only)
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

## 2. Building Your A2A System

Let's build our three-agent system step by step. We'll create:

1. **Trending Topics Agent** - Finds current trending topics
2. **Trend Analyzer Agent** - Analyzes trends with quantitative data
3. **Host Agent** - Orchestrates the other agents

### Agent 1: Trending Topics Agent

This agent searches the web for trending topics and returns a list of current trends.

In [4]:
from google.adk.agents import Agent
from google.adk.tools import google_search

# Create the Trending Topics ADK Agent
trending_agent = Agent(
    model="gemini-2.5-pro",
    name="trending_topics_agent",
    instruction="""
    You are a social media trends analyst. Your job is to search the web for current trending topics,
    particularly from social platforms.
    
    When asked about trends:
    1. Search for "trending topics today" or similar queries
    2. Extract the top 3 trending topics
    3. Return them in a JSON format
    
    Focus on current, real-time trends from the last 24 hours.
    
    You MUST return your response in the following JSON format:
    {
        "trends": [
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)", 
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            }
        ]
    }
    
    Only return the JSON object, no additional text.
    """,
    tools=[google_search],
)

print("Trending Topics Agent created successfully!")

Trending Topics Agent created successfully!


### Agent 2: Trend Analyzer Agent

This agent takes a specific trend and performs deep analysis with quantitative data.

In [5]:
from google.adk.agents import Agent
from google.adk.tools import google_search

# Create the Trend Analyzer ADK Agent
analyzer_agent = Agent(
    model="gemini-2.5-pro",
    name="trend_analyzer_agent",
    instruction="""
    You are a data analyst specializing in trend analysis. When given a trending topic,
    perform deep research to find quantitative data and insights.
    
    For each trend you analyze:
    1. Search for statistics, numbers, and metrics related to the trend
    2. Look for:
       - Engagement metrics (views, shares, mentions)
       - Growth rates and timeline
       - Geographic distribution
       - Related hashtags or keywords
    3. Provide concrete numbers and data points

    Keep it somehow concise
    
    Always prioritize quantitative information over qualitative descriptions.
    """,
    tools=[google_search],
)

print("Trend Analyzer Agent created successfully!")

Trend Analyzer Agent created successfully!


### A2A Server Wrapper

Now let's create the A2A server wrapper that exposes our ADK agents via the A2A protocol:

In [6]:
import asyncio
import base64
import uvicorn
from a2a.types import AgentSkill
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore, TaskUpdater
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.types import (
    AgentCard,
    AgentCapabilities,
    DataPart,
    Message,
    MessageSendConfiguration,
    MessageSendParams,
    Part,
    Task,
    TaskState,
    TextPart,
)
from a2a.utils import new_agent_text_message, new_task
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Generic A2A Executor for any ADK agent
class ADKAgentExecutor(AgentExecutor):
    def __init__(self, agent, status_message="Processing request...", artifact_name="response"):
        """
        Initialize a generic ADK agent executor.
        
        Args:
            agent: The ADK agent instance
            status_message: Message to display while processing
            artifact_name: Name for the response artifact
        """
        self.agent = agent
        self.status_message = status_message
        self.artifact_name = artifact_name
        self.runner = Runner(
            app_name=agent.name,
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
    
    async def cancel(self, task_id: str) -> None:
        """Cancel the execution of a specific task."""
        # Implementation for cancelling tasks
        pass
    
    async def execute(self, context: RequestContext, event_queue: EventQueue) -> None:
        query = context.get_user_input()
        task = context.current_task or new_task(context.message)
        await event_queue.enqueue_event(task)
        
        updater = TaskUpdater(event_queue, task.id, task.contextId)
        
        try:
            # Update status with custom message
            await updater.update_status(
                TaskState.working,
                new_agent_text_message(self.status_message, task.contextId, task.id)
            )
            
            # Process with ADK agent
            session = await self.runner.session_service.create_session(
                app_name=self.agent.name,
                user_id="a2a_user",
                state={},
                session_id=task.contextId,
            )
            
            content = types.Content(
                role='user',
                parts=[types.Part.from_text(text=query)]
            )
            
            response_text = ""
            async for event in self.runner.run_async(
                user_id="a2a_user",
                session_id=session.id,
                new_message=content
            ):
                if event.is_final_response() and event.content and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            response_text += part.text + '\n'
                        elif hasattr(part, 'function_call'):
                            # Log or handle function calls if needed
                            pass  # Function calls are handled internally by ADK
            
            # Add response as artifact with custom name
            await updater.add_artifact(
                [Part(root=TextPart(text=response_text))],
                name=self.artifact_name
            )
            
            await updater.complete()
            
        except Exception as e:
            await updater.update_status(
                TaskState.failed,
                new_agent_text_message(f"Error: {str(e)}", task.contextId, task.id),
                final=True
            )

# Generic function to create an A2A server for any ADK agent
def create_agent_a2a_server(
    agent, 
    name,
    description,
    skills,
    host="localhost",
    port=10020,
    status_message="Processing request...",
    artifact_name="response"
):
    """
    Create an A2A server for any ADK agent.
    
    Args:
        agent: The ADK agent instance
        name: Display name for the agent
        description: Agent description
        skills: List of AgentSkill objects
        host: Server host
        port: Server port
        status_message: Message shown while processing
        artifact_name: Name for response artifacts
    
    Returns:
        A2AStarletteApplication instance
    """
    # Agent capabilities
    capabilities = AgentCapabilities(streaming=True)
    
    # Agent card (metadata)
    agent_card = AgentCard(
        name=name,
        description=description,
        url=f"http://{host}:{port}/",
        version="1.0.0",
        defaultInputModes=["text", "text/plain"],
        defaultOutputModes=["text", "text/plain"],
        capabilities=capabilities,
        skills=skills,
    )
    
    # Create executor with custom parameters
    executor = ADKAgentExecutor(
        agent=agent,
        status_message=status_message,
        artifact_name=artifact_name
    )
    
    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )
    
    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler
    )

### A2A Server for Analyzer Agent

In [7]:
def create_trending_agent_server(host="localhost", port=10020):
    """Create A2A server for Trending Agent using the unified wrapper."""
    return create_agent_a2a_server(
        agent=trending_agent,
        name="Trending Topics Agent",
        description="Searches the web for current trending topics from social media",
        skills=[
            AgentSkill(
                id="find_trends",
                name="Find Trending Topics",
                description="Searches for current trending topics on social media",
                tags=["trends", "social media", "twitter", "current events"],
                examples=[
                    "What's trending today?",
                    "Show me current Twitter trends",
                    "What are people talking about on social media?",
                ],
            )
        ],
        host=host,
        port=port,
        status_message="Searching for trending topics...",
        artifact_name="trending_topics"
    )

### A2A Server for Trending Agent

In [8]:
def create_analyzer_agent_server(host="localhost", port=10021):
    """Create A2A server for Analyzer Agent using the unified wrapper."""
    return create_agent_a2a_server(
        agent=analyzer_agent,
        name="Trend Analyzer Agent",
        description="Performs deep analysis of trends with quantitative data",
        skills=[
            AgentSkill(
                id="analyze_trend",
                name="Analyze Trend",
                description="Provides quantitative analysis of a specific trend",
                tags=["analysis", "data", "metrics", "statistics"],
                examples=[
                    "Analyze the #ClimateChange trend",
                    "Get metrics for the Taylor Swift trend",
                    "Provide data analysis for AI adoption trend",
                ],
            )
        ],
        host=host,
        port=port,
        status_message="Analyzing trend with quantitative data...",
        artifact_name="trend_analysis"
    )

### A2AToolClient for the Host Agent (Orchestrator) to use and communicate with remote agents

In [9]:
import requests
import json
import uuid
from typing import List, Dict, Any, Optional
from a2a.client import A2AClient
from a2a.types import SendMessageRequest, MessageSendParams, AgentCard
import httpx

class A2AToolClient:
    """A2A client."""
    
    def __init__(self, default_timeout: float = 120.0):
        # Cache for agent metadata - also serves as the list of registered agents
        # None value indicates agent is registered but metadata not yet fetched
        self._agent_info_cache: Dict[str, Optional[Dict[str, Any]]] = {}
        # Default timeout for requests (in seconds)
        self.default_timeout = default_timeout
    
    def add_remote_agent(self, agent_url: str):
        """Add agent to the list of available remote agents."""
        normalized_url = agent_url.rstrip('/')
        if normalized_url not in self._agent_info_cache:
            # Initialize with None to indicate metadata not yet fetched
            self._agent_info_cache[normalized_url] = None
            
    def list_remote_agents(self) -> List[Dict[str, Any]]:
        """List available remote agents with caching."""
        if not self._agent_info_cache:
            return []
        
        remote_agents_info = []
        for remote_connection in self._agent_info_cache:
            # Use cached data if available
            if self._agent_info_cache[remote_connection] is not None:
                remote_agents_info.append(self._agent_info_cache[remote_connection])
            else:
                try:
                    # Fetch and cache agent info
                    agent_info = requests.get(f"{remote_connection}/.well-known/agent.json")
                    agent_data = agent_info.json()
                    self._agent_info_cache[remote_connection] = agent_data
                    remote_agents_info.append(agent_data)
                except Exception as e:
                    print(f"Failed to fetch agent info from {remote_connection}: {e}")
        
        return self._agent_info_cache
    
    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0, 
            read=self.default_timeout,
            write=10.0, 
            pool=5.0
        )
        
        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if agent_url in self._agent_info_cache and self._agent_info_cache[agent_url] is not None:
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(f"{agent_url}/.well-known/agent.json")
                agent_card_data = agent_card_response.json()
            
            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)
            
            # Create A2A client with the agent card
            client = A2AClient(
                httpx_client=httpx_client,
                agent_card=agent_card
            )
            
            # Build the message parameters following official structure
            send_message_payload = {
                'message': {
                    'role': 'user',
                    'parts': [
                        {'kind': 'text', 'text': message}
                    ],
                    'messageId': uuid.uuid4().hex,
                }
            }
            
            # Create the request
            request = SendMessageRequest(
                id=str(uuid.uuid4()),
                params=MessageSendParams(**send_message_payload)
            )
            
            # Send the message with timeout configuration
            response = await client.send_message(request)
            
            # Extract text from response
            try:
                response_dict = response.model_dump(mode='json', exclude_none=True)
                if 'result' in response_dict and 'artifacts' in response_dict['result']:
                    artifacts = response_dict['result']['artifacts']
                    for artifact in artifacts:
                        if 'parts' in artifact:
                                for part in artifact['parts']:
                                    if 'text' in part:
                                        return part['text']
                
                # If we couldn't extract text, return the full response as formatted JSON
                return json.dumps(response_dict, indent=2)
                
            except Exception as e:
                # Log the error and return string representation
                print(f"Error parsing response: {e}")
                return str(response)
    
    def remove_remote_agent(self, agent_url: str):
        """Remove an agent from the list of available remote agents."""
        normalized_url = agent_url.rstrip('/')
        if normalized_url in self._agent_info_cache:
            del self._agent_info_cache[normalized_url]


In [10]:
a2a_client = A2AToolClient()

### Agent 3: Host Agent (Orchestrator)

The Host Agent coordinates between the other two agents to provide comprehensive trend analysis.

In [11]:
# Create the Host ADK Agent
host_agent = Agent(
    model="gemini-2.5-pro",
    name="trend_analysis_host",
    instruction="""
You are an expert AI Orchestrator.
Your primary responsibility is to intelligently interpret user requests, plan the necessary sequence of actions if multiple steps are involved, and delegate them to the most appropriate specialized remote agents.
You do not perform the tasks yourself but manage their assignment, sequence, and can monitor their status.

Core Workflow & Decision Making:

1.  **Understand User Intent & Complexity:**
    *   Carefully analyze the user's request to determine the core task(s) they want to achieve. Pay close attention to keywords and the overall goal.
    *   **Identify if the request requires a single agent or a sequence of actions from multiple agents.** For example, it could require two agents to be called in sequence.

2.  **Agent Discovery & Selection:**
    *   You must start by using the `list_remote_agents` tool to get an up-to-date dictionary (urls as keys) of available remote agents and understand their specific capabilities (e.g., what kind of requests each agent is designed to handle and what data they output).
    *   Based on the user's intent:
        *   For **single-step requests**, select the single most appropriate agent.
        *   For **multi-step requests**, identify all necessary agents and determine the logical order of their execution.

3.  **Task Planning & Sequencing (for Multi-Step Requests):**
    *   Before delegating, outline the sequence of agent tasks.
    *   Identify dependencies: Does Agent B need information from Agent A's completed task?
    *   Plan to execute tasks sequentially if there are dependencies, waiting for the completion of a prerequisite task before initiating the next one.

4.  **Task Delegation & Management:**
    *   **For New Single Requests or the First Step in a Sequence:** Use `create_task`. Your `create_task` call MUST include:
        *   The `agent_url` as the `url` parameter extracted from the remote agent you've selected from the dictionary of available remote agents (url is the key), ex: http://localhost:10021
        *   The `message` extracted from the user's input, formatted in a way the target agent will understand (check the agent info to better structure the message). 
    *   **For Subsequent Steps in a Sequence:**
        *   Once the prerequisite task is done, gather any necessary output from it.
        *   Then, use `create_task` for the next agent in the sequence, providing it with the user's original relevant intent and any necessary data obtained from the previous agent's task.

**Communication with User:**

*   Clearly inform the user which agent is handling each task. The user should know the entire sequence of agents you used and the results of each one.
*   If the user's request is ambiguous, if necessary information is missing for any agent in the sequence, or if you are unsure about the plan, proactively ask the user for clarification.
*   Rely strictly on your tools and the information they provide.
*   Communicate to the user the content from the data gathered from the all remote agents responses.
*   The communication to the user should contain most of the information from the remote agents, do not summarize too much.

**Important Reminders:**
*   Always prioritize selecting the correct agent(s) based on their documented purpose.
*   Ensure all information required by the chosen remote agent is included in the `create_task` call, including outputs from previous agents if it's a sequential task.
*   Focus on the most recent parts of the conversation for immediate context, but maintain awareness of the overall goal, especially for multi-step requests.

""",
    tools=[a2a_client.list_remote_agents, a2a_client.create_task]
)

### A2A Server for Host Agent

In [12]:
def create_host_agent_server(host="localhost", port=10022):
    """Create A2A server for Host Agent using the unified wrapper."""
    return create_agent_a2a_server(
        agent=host_agent,
        name="Trend Analysis Host",
        description="Orchestrates trend discovery and analysis using specialized agents",
        skills=[
            AgentSkill(
                id="comprehensive_trend_analysis",
                name="Comprehensive Trend Analysis",
                description="Finds trending topics and provides deep analysis of the most relevant one",
                tags=["trends", "analysis", "orchestration", "insights"],
                examples=[
                    "Analyze current trends",
                    "What's trending and why is it important?",
                    "Give me a comprehensive trend report",
                ],
            )
        ],
        host=host,
        port=port,
        status_message="Orchestrating trend analysis...",
        artifact_name="trend_report"
)

## 3. Running

Now let's put everything together. We'll create helper functions to start our agents and run the complete system.

### Starting the A2A Servers

Create functions to run each agent as an A2A server:

In [13]:
import threading
import asyncio
import time
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Global servers to keep references
servers = []

async def run_server_notebook(create_agent_function, port):
    """Run server with proper error handling."""
    try:
        print(f"🚀 Starting agent on port {port}...")
        app = create_agent_function()
        config = uvicorn.Config(
            app.build(), 
            host="127.0.0.1",  
            port=port, 
            log_level="error",  
            loop="asyncio"
        )
        server = uvicorn.Server(config)
        servers.append(server)
        await server.serve()
    except Exception as e:
        print(f"Agent error: {e}")

def run_agent_in_background(create_agent_function, port, name):
    """Run an agent server in a background thread."""
    def run():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        try:
            # Create the coroutine inside the new event loop
            loop.run_until_complete(run_server_notebook(create_agent_function, port))
        except Exception as e:
            print(f"{name} error: {e}")
    
    thread = threading.Thread(target=run, daemon=True)
    thread.start()
    return thread

# Start agent servers with corrected function calls
print("Starting agent servers...\n")

trending_thread = run_agent_in_background(create_trending_agent_server, 10020, "Trending Agent")
analyzer_thread = run_agent_in_background(create_analyzer_agent_server, 10021, "Analyzer Agent")
host_thread = run_agent_in_background(create_host_agent_server, 10022, "Host Agent")

# Wait for servers to start
time.sleep(3)

# Check if threads are alive
if trending_thread.is_alive() and analyzer_thread.is_alive():
    print("\n✅ Agent servers are running!")
    print("   - Trending Agent: http://127.0.0.1:10020")
    print("   - Analyzer Agent: http://127.0.0.1:10021")
    print("   - Host Agent: http://127.0.0.1:10022")
else:
    print("\n❌ Agent servers failed to start. Check the error messages above.")

Starting agent servers...

🚀 Starting agent on port 10020...
🚀 Starting agent on port 10022...
🚀 Starting agent on port 10021...

✅ Agent servers are running!
   - Trending Agent: http://127.0.0.1:10020
   - Analyzer Agent: http://127.0.0.1:10021
   - Host Agent: http://127.0.0.1:10022


## 4. Testing the System

Let's test our A2A system with Python requests.

### Test Agent Discovery

In [14]:
a2a_client.add_remote_agent("http://localhost:10020")
a2a_client.add_remote_agent("http://localhost:10021")

remote_agents = a2a_client.list_remote_agents()
for k, v in remote_agents.items():
    print(f"Remote agent url: {k}")
    print(f"Remote agent name: {v['name']}")
    print(f"Remote agent skills: {v['skills']}")
    print(f"Remote agent version: {v['version']}")
    print(f"----\n")

Remote agent url: http://localhost:10020
Remote agent name: Trending Topics Agent
Remote agent skills: [{'description': 'Searches for current trending topics on social media', 'examples': ["What's trending today?", 'Show me current Twitter trends', 'What are people talking about on social media?'], 'id': 'find_trends', 'name': 'Find Trending Topics', 'tags': ['trends', 'social media', 'twitter', 'current events']}]
Remote agent version: 1.0.0
----

Remote agent url: http://localhost:10021
Remote agent name: Trend Analyzer Agent
Remote agent skills: [{'description': 'Provides quantitative analysis of a specific trend', 'examples': ['Analyze the #ClimateChange trend', 'Get metrics for the Taylor Swift trend', 'Provide data analysis for AI adoption trend'], 'id': 'analyze_trend', 'name': 'Analyze Trend', 'tags': ['analysis', 'data', 'metrics', 'statistics']}]
Remote agent version: 1.0.0
----



### Test Direct Agent Communication

In [15]:
trending_topics = await a2a_client.create_task("http://localhost:10020", "What's trending today?")
print(trending_topics)

{
    "trends": [
        {
            "topic": "Trump Family Mobile Phone Company",
            "description": "The Trump family's company has announced the launch of a new mobile phone company, with a reported price of $499 for the cellphone.",
            "reason": "This is trending due to the high public profile of the Trump family and the novelty of them entering the technology and telecommunications market."
        },
        {
            "topic": "Matthew Perry's Death Investigation",
            "description": "A second doctor is set to plead guilty to charges related to supplying ketamine to the late actor Matthew Perry, who fatally overdosed in 2023.",
            "reason": "The ongoing investigation into the high-profile death of the beloved 'Friends' actor continues to draw significant public interest and media coverage."
        },
        {
            "topic": "Venice Protests Against Jeff Bezos' Wedding",
            "description": "Locals in Venice are protesting th

In [16]:
analysis = await a2a_client.create_task("http://localhost:10021", "Analyze the trend AI in Social Media")
print(analysis)

## The Meteoric Rise of AI in Social Media: A Data-Driven Analysis

The integration of artificial intelligence into social media has become a dominant and rapidly expanding trend, transforming how users interact with content and how businesses conduct their marketing. Quantitative data reveals a significant and accelerating adoption of AI technologies across all facets of social media, from content creation and personalization to market growth and user engagement.

**Market Growth and Projections:** The global market for AI in social media is experiencing explosive growth. Valued at approximately USD 1.93 billion to USD 2.05 billion in 2023 and 2024 respectively, it is projected to skyrocket to between USD 11.99 billion and USD 22.30 billion by the early 2030s. This expansion is underscored by a compound annual growth rate (CAGR) estimated to be between 12.75% and 31.4%.

**AI's Deep Integration into Social Platforms:** Artificial intelligence is now a fundamental component of the soci

### Run the Complete Host Agent

In [17]:
host_analysis = await a2a_client.create_task("http://localhost:10022", "Find the most relevant trends in the web today, choose randomly one of the top trends, and give me a complete analysis of it with quantitative data")
print(host_analysis)

Here is a complete analysis of the **G7 Summit 2025** trend, with quantitative data, as provided by the Trend Analyzer Agent:

### G7 Summit 2025: A Data-Driven Analysis of an Evolving Global Agenda

The 2025 G7 Summit, hosted by Canada in Kananaskis, Alberta from June 15-17, convenes at a critical juncture for the world's leading economies. Against a backdrop of uneven economic recovery, escalating geopolitical tensions, and the accelerating challenges of climate change and technological disruption, the summit's agenda is squarely focused on three pillars: safeguarding global security, advancing the energy and digital transitions, and forging resilient future partnerships. This analysis delves into the quantitative data underpinning these priorities, offering a snapshot of the trends and challenges shaping the G7's discussions.

**The Economic Landscape: A Mixed Picture of Growth and Strain**

The G7 nations, which collectively accounted for about 25.4% of global GDP in 2024, are navi

## Summary

Congratulations! You've successfully built a multi-agent system using Google's A2A protocol. Here's what you've learned:

1. **A2A Protocol Basics**: How agents discover and communicate with each other
2. **ADK Integration**: Creating ADK agents and wrapping them for A2A
3. **Agent Orchestration**: Building a Host Agent that coordinates multiple agents
4. **Practical Implementation**: Running and testing a complete multi-agent system

### Next Steps

- **Deploy Your Agents**: Deploy agents to Cloud Run or other platforms
- **Add Authentication**: Implement security for production use
- **Create More Agents**: Build agents for your specific use cases, even using other frameworks
- **Advanced Patterns**: Explore agent chains, parallel execution, and more
- **Callbacks**: Add in the Google ADK agents the before and after callbacks of the agent, model and tool, to increase observability

Happy agent building! 🚀

# Appendix

### Why Use Google A2A (Agent-to-Agent) Protocol

Google's Agent-to-Agent (A2A) protocol is a standardized communication framework that enables AI agents to discover, communicate, and collaborate with each other using a common JSON-RPC based protocol.  
It provides a uniform way for agents to interact, regardless of their underlying implementation.  

#### 1. Standardized Communication Protocol

- A2A provides a consistent, JSON-RPC based protocol that any agent can implement
- Agents can communicate without needing to know each other's internal implementation details
- The protocol supports streaming responses for real-time interactions

#### 2. Agent Discovery and Metadata

- Agents expose their capabilities through standardized metadata (AgentCard)
- Each agent publishes its skills, input/output modes, and capabilities
- Host agents can dynamically discover what other agents can do through the `.well-known/agent.json` endpoint

#### 3. Orchestration and Composition

- Enables building complex multi-agent systems where a host agent can orchestrate multiple specialized agents
- Supports sequential and parallel task execution patterns
- Allows for sophisticated agent collaboration workflows

#### 4. Platform Independence

- A2A servers can wrap agents from different frameworks (not just ADK)
- Agents can be deployed as independent services on different infrastructure
- Promotes loose coupling between agents

### Differences: Using ADK Agents Directly vs. Through A2A

#### Using ADK Agents Directly

```python
# Conceptual Example: Defining Hierarchy
from google.adk.agents import LlmAgent, BaseAgent

# Define individual agents
greeter = LlmAgent(name="Greeter", model="gemini-2.5-pro")
task_doer = BaseAgent(name="TaskExecutor") # Custom non-LLM agent

# Create parent agent and assign children via sub_agents
coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.5-pro",
    description="I coordinate greetings and tasks.",
    sub_agents=[ # Assign sub_agents here
        greeter,
        task_doer
    ]
)
```

__Use Direct ADK for Multi-Agents System When:__

- All agents are tightly related and always used together
- Google ADK is the framework choice, and simplicity is prioritized
- Performance of in-process communication is critical
- You don't need distributed deployment
- No built-in service discovery is needed

#### Using ADK Agents Through A2A

__Use A2A for Multi-Agents System When:__

- Building complex multi-agent systems
- Agents need to be developed, deployed, and scaled independently
- You want to integrate agents from different teams or frameworks
- You need dynamic agent discovery and composition
- Building a platform where agents can be added/removed dynamically
- You want to enable third-party agent integration